# MNISTをCSVに変換
[struct](https://docs.python.jp/3/library/struct.html)はバイナリ処理するためのモジュール。開くときは"r"ではなく"br"で開く。  
MNISTのデータベースファイルは、６万枚の画像情報だけを保存した「*-images-idx3-ubyte」とその画像がどの数字を表すのかを保存した「*-labels-idx1-ubyte」のセットで成り立つ。  
## *1
ヘッダー情報を読む。それぞれデータは[ビッグエンディアン](http://e-words.jp/w/リトルエンディアン.html)になっている。ビッグエンディアンのデータを読み出すには「>」の記号を指定する。マジックナンバーを表す32bitと、画像枚数を表す32bit整数を読み出す。画像データベースでは、続く８バイトに、画像の横ピクセル数と縦ピクセル数の情報が入る。

In [6]:
import struct

def to_csv(name, maxdata):
    # ラベルファイルとイメージファイルを開く
    lbl_f = open("./mnist/"+name+"-labels-idx1-ubyte", "rb")
    img_f = open("./mnist/"+name+"-images-idx3-ubyte", "rb")
    csv_f = open("./mnist/"+name+".csv", "w", encoding="utf-8")
    # ヘッダ情報を読む --- (※1)
    mag, lbl_count = struct.unpack(">II", lbl_f.read(8))
    mag, img_count = struct.unpack(">II", img_f.read(8))
    rows, cols = struct.unpack(">II", img_f.read(8))
    pixels = rows * cols
    # 画像データを読んでCSVで保存 --- (※2)
    res = []
    for idx in range(lbl_count):
        if idx > maxdata: break
        # バイト数を読み込んで、整数として読み込む
        label = struct.unpack("B", lbl_f.read(1))[0]
        bdata = img_f.read(pixels)
        sdata = list(map(lambda n: str(n), bdata))
        csv_f.write(str(label)+",")
        csv_f.write(",".join(sdata)+"\r\n")
        # うまく取り出せたかどうかPGMで保存してテスト -- (*3)
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = "./mnist/{0}-{1}-{2}.pgm".format(name,idx,label)
            with open(iname, "w", encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()

# 出力件数を指定 --- (※4)
# 学習用
to_csv("train", 1000)
# テスト用
to_csv("t10k", 500)